In [1]:
import sys
src_dir = '/Users/palmerio/Science_projects/GRB_population_model/src/'
sys.path.insert(0, src_dir)
verbose = True
run_mode = 'debug'
# import prototype_GRB_population as pt
from GRB_population import GRBPopulation
import physics as ph
import numpy as np
import io_grb_pop as io
import miscellaneous as msc
from cosmology import init_cosmology
from ECLAIRs import init_ECLAIRs
import observational_constraints as obs

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('presentation')


In [3]:
paths_to_dir, paths_to_files = io.generate_paths()

In [5]:
config, params, instruments, samples, obs_constraints = io.read_init_files(paths_to_files)
config['Nb_GRBs'] = '1e5'
config['samples'] = ['Stern', 'EpGBM', 'eBAT6']
config['output_dir'] = 'debugger'
paths_to_dir['output'] = paths_to_dir['output']/config['output_dir']

In [6]:
print(paths_to_dir['output'])

/Users/palmerio/Science_projects/GRB_population_model/model_outputs/debugger/debugger


In [7]:
cosmo = init_cosmology(paths_to_dir['cosmo'])
ECLAIRs_prop = init_ECLAIRs(ECLAIRs_dir=paths_to_dir['ECLAIRs'],
                            ECLAIRs_config=instruments['ECLAIRs'])
samples['ECLAIRs']['pflx_min'] = ECLAIRs_prop['bkg_total']

Nb_GRBs = int(float(config['Nb_GRBs']))

obs_constraints = obs.load_observational_constraints(obs_constraints)
incl_samples = msc.included_samples(config['samples'], samples)
incl_instruments = msc.included_instruments(incl_samples, instruments)

ValueError: Constraint name must be one of ['Stern', 'EpGBM', 'eBAT6']

In [ ]:
np.random.seed(1)
GRB_population = GRBPopulation(Nb_GRBs, output_dir=paths_to_dir['output'])
GRB_prop = GRB_population.draw_GRB_properties(cosmo=cosmo, params=params, run_mode='debug')

ph.calc_peak_photon_flux(GRB_prop, incl_instruments, ECLAIRs_prop)
# ph.calc_peak_energy_flux(GRB_prop, incl_instruments, ECLAIRs_prop)
# ph.calc_photon_fluence(GRB_prop, incl_instruments)
# ph.calc_energy_fluence(GRB_prop, incl_instruments)
ph.calc_det_prob(GRB_prop, incl_samples, **ECLAIRs_prop)

In [ ]:
df = pd.DataFrame(GRB_prop)
df

In [ ]:
norm_Stern, mod_Stern = obs.compare_to_Stern(df[df['pdet_Stern']==1]['pht_pflx_BATSE'],
                         Stern_file=paths_to_dir['obs']/'Stern_lognlogp_rebinned.txt',
                         Nb_GRBs=Nb_GRBs, show_plot=True)

In [ ]:
obs.compare_to_EpGBM(df[df['pdet_EpGBM']==1]['Epobs'],
                         GBM_file=paths_to_dir['obs']/'EpGBM_for_plotting.txt',
                        show_plot=True)


In [ ]:
obs.compare_to_eBAT6(df[df['pdet_eBAT6']==1]['z'],
                     eBAT6_file=paths_to_dir['obs']/'eBAT6_constraint_for_plotting.txt',
                     show_plot=True)


In [ ]:
f90_dir = paths_to_dir['output'].parent/'191205_best_k1_LN_nF'


In [ ]:
dt = 'i4,i4,12f8,11f8,i4'
# i, L, z, D_L, Epobs, alpha, beta, ktild, Peakflux_Instrument, Prob_det
with open(f90_dir/'GRB_properties 0.dat', 'rb') as f:
    data = np.fromfile(f, dtype=dt, count=-1)
df_f90 = np.zeros((len(data),12))
for i,dat in enumerate(data):
    df_f90[i] = dat[2]

df_f90 = pd.DataFrame(df_f90, columns=['L', 'z', 'D_L', 'Epobs', 'alpha', 'beta', 'ktild',
                                     'pht_pflx_BATSE','pht_pflx_BAT','pht_pflx_FREGATE','pht_pflx_WXM','pht_pflx_ECLAIRs'])
# df_f90['L'] = np.log10(df_f90['L'])

In [ ]:
norm_Stern, mod_Stern = obs.compare_to_Stern(df_f90[df_f90['pht_pflx_BATSE']>=0.066825]['pht_pflx_BATSE'],
                         Stern_file=paths_to_dir['obs']/'Stern_lognlogp_rebinned.txt',
                         Nb_GRBs=Nb_GRBs, show_plot=True)

In [ ]:
obs_constraints['Stern']['hist']